## Librerias

In [7]:
import generators as gen 
import os 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chisquare, pearsonr, kstest

## Funciones auxiliares

In [8]:
""" Funciones para plot """
def ensure_dir(directory):
    os.makedirs(directory, exist_ok=True)

# Función para plot 3D y guardar automáticamente
def plot_3d(points, title='', save_dir='plots', filename=None):
    ensure_dir(save_dir)
    if filename is None:
        filename = f"plot3d_{title.replace(' ', '_') or 'untitled'}.png"

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=1, alpha=0.6)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_title(title)
    save_path = os.path.join(save_dir, filename)
    plt.savefig(save_path, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved 3D plot to {save_path}")

# Función para scatter de pares y guardar automáticamente
def plot_pairs(points, title='', save_dir='plots', filename=None):
    ensure_dir(save_dir)
    if filename is None:
        filename = f"plot_pairs_{title.replace(' ', '_') or 'untitled'}.png"

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(points[:, 0], points[:, 1], s=1, alpha=0.6)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title(title)
    ax.grid(True)
    save_path = os.path.join(save_dir, filename)
    plt.savefig(save_path, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved pairs plot to {save_path}")

# Función para histograma y guardar automáticamente
def plot_histogram(values, bins=50, title='', save_dir='plots', filename=None):
    ensure_dir(save_dir)
    if filename is None:
        filename = f"histogram_{title.replace(' ', '_') or 'untitled'}.png"

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.hist(values, bins=bins, density=True, alpha=0.7)
    ax.set_xlabel("Value")
    ax.set_ylabel("Density")
    ax.set_title(title)
    ax.grid(True)
    save_path = os.path.join(save_dir, filename)
    plt.savefig(save_path, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved histogram to {save_path}")


""" Funciones que permiten el análisis estadístico """

def generate_d_points(rng, d):
    n = len(rng) // d
    return np.array(rng[:n*d]).reshape((n, d))


def chi_square_uniformity(rng, n=10000, bins=60):
    values = np.array([next(rng) for _ in range(n)])
    counts, _ = np.histogram(values, bins=bins, range=(0, 1))
    expected = np.full(bins, n / bins)
    if np.any(expected < 5):
        raise ValueError("Some expected frequencies are below 5. Consider reducing the number of bins.")
    stat, p_value = chisquare(counts, expected)
    return stat, p_value

def correlation_test(rng, n=10000, lag=1):
    values = np.array([next(rng) for _ in range(n + lag)])
    x = values[:-lag]
    y = values[lag:]
    corr, p_value = pearsonr(x, y)
    return corr, p_value


def ks_test_uniformity(rng, n=10000):
    values = np.array([next(rng) for _ in range(n)])
    stat, p_value = kstest(values, 'uniform', args=(0, 1))
    return stat, p_value

## Tests estadísticos


In [ ]:
N_test = [10000, 20000, 30000, 40000, 50000, 60000]
bins = [60, 100, 150, 200, 250, 300]
chi_tests: list[tuple[str, float, float]] = []
autocorr_tests: list[tuple[str, float, float]] = []
ks_test: list[tuple[str, float, float]] = []

print("Resultados del test Chi-cuadrado para uniformidad:")
for n_test in N_test:
    print(f"\nTest con {n_test} muestras:")
    for bin in bins:
        print(f"\nTomando {bin} bins:")
        genCong = gen.genCongruencial(2605, 16807, 0, 2**31 - 1)
        genXor = gen.XorShift(2463534242)
        genMersenne = gen.Mersenne(5489)


        print("\nChi-cuadrado (Uniformidad):")
        for name, rng in zip(["LCG", "XorShift", "Mersenne"], [genCong, genXor, genMersenne]):
            stat, p = chi_square_uniformity(rng, n=n_test, bins=bin)
            chi_tests.append((name, stat, p))
            print(f"{name}: stat = {stat:.4f}, p-value = {p:.4f}")


print("\nPromedio de p-values para cada generador en el test Chi-cuadrado:")
avg_p_values = {}
for name, stat, p in chi_tests:
    if name not in avg_p_values:
        avg_p_values[name] = []
    avg_p_values[name].append(p)

for name, p_values in avg_p_values.items():
    avg_p = np.mean(p_values)
    print(f"\nPromedio de p-value para {name}: {avg_p:.4f}")


print("\nResultados del test de autocorrelación:")
for n_test in N_test:
    genCong = gen.genCongruencial(2605, 16807, 0, 2**31 - 1)
    genXor = gen.XorShift(2463534242)
    genMersenne = gen.Mersenne(5489)

    print(f"\nAutocorrelación para {n_test} muestras:")
    for name, rng in zip(["LCG", "XorShift", "Mersenne"], [genCong, genXor, genMersenne]):
        for lgs in [1, 2, 5, 10, 100, int((n_test * 0.1)/2), int(n_test * 0.1)]:
            print(f"\nLag = {lgs}:")
            r, p_value = correlation_test(rng, n=n_test, lag=lgs)
            autocorr_tests.append((name, r, p_value))
            print(f"{name}: r = {r:.4f}, p-value = {p_value:.4f}")

print("\nPromedio de correlación para cada generador en el test de autocorrelación:")
avg_corr_values = {}
for name, r, p in autocorr_tests:
    if name not in avg_corr_values:
        avg_corr_values[name] = []
    avg_corr_values[name].append(r)
for name, r_values in avg_corr_values.items():
    avg_r = np.mean(r_values)
    print(f"\nPromedio de r para {name}: {avg_r:.4f}")


print("\nResultados del test de Kolmogorov-Smirnov para uniformidad:")
for n in N_test:
    genCong = gen.genCongruencial(2605, 16807, 0, 2**31 - 1)
    genXor = gen.XorShift(2463534242)
    genMersenne = gen.Mersenne(5489)

    print(f"\nKolmogorov-Smirnov para {n} muestras:")
    for name, rng in zip(["LCG", "XorShift", "Mersenne"], [genCong, genXor, genMersenne]):
        stat, p_value = ks_test_uniformity(rng, n=n)
        ks_test.append((name, stat, p_value))
        print(f"{name}: stat = {stat:.4f}, p-value = {p_value:.4f}")
        
print("\nPromedio de p-values para cada generador en el test de Kolmogorov-Smirnov:")
avg_p_values_ks = {}
for name, stat, p in ks_test:
    if name not in avg_p_values_ks:
        avg_p_values_ks[name] = []
    avg_p_values_ks[name].append(p)
for name, p_values in avg_p_values_ks.items():
    avg_p = np.mean(p_values)
    print(f"\nPromedio de p-value para {name}: {avg_p:.4f}")

## Grafico en 2 y 3 Dimensiones, Histograma de valores
Adaptar correctamente el directorio en `base_dir`para correr el bloque siguiente.

In [ ]:
base_dir = "./" 

points_per_dim = [10000, 40000]

for points in points_per_dim:
    # Histograma de valores generados por los RNGs
    N = points
    genCong = gen.genCongruencial(2605, 16807, 0, 2**31 - 1)
    genXor = gen.XorShift(2463534242)
    genMersenne = gen.Mersenne(5489)

    lcg_rng = [next(genCong) for _ in range(N)]
    xor_rng = [next(genXor) for _ in range(N)]
    mt_rng = [next(genMersenne) for _ in range(N)]

    plot_histogram(lcg_rng,
                   bins=50,
                   title=f'Histograma_LCG_{N}',
                   save_dir=base_dir,
                   filename=f"hist_LCG_{N}.png")
    plot_histogram(xor_rng,
                   bins=50,
                   title=f'Histograma_XorShift_{N}',
                   save_dir=base_dir,
                   filename=f"hist_XorShift_{N}.png")
    plot_histogram(mt_rng,
                   bins=50,
                   title=f'Histograma_Mersenne_{N}',
                   save_dir=base_dir,
                   filename=f"hist_Mersenne_{N}.png")


    # Puntos 2D
    M = points * 2
    genCong = gen.genCongruencial(2605, 16807, 0, 2**31 - 1)
    genXor = gen.XorShift(2463534242)
    genMersenne = gen.Mersenne(5489)

    lcg_rng = [next(genCong) for _ in range(M)]
    xor_rng = [next(genXor) for _ in range(M)]
    mt_rng = [next(genMersenne) for _ in range(M)]

    points_lcg = generate_d_points(lcg_rng, 2)
    points_xor = generate_d_points(xor_rng, 2)
    points_mt  = generate_d_points(mt_rng, 2)

    plot_pairs(points_lcg,
               title=f'Puntos_LCG_2D_{N}',
               save_dir=base_dir,
               filename=f"pairs_LCG_{N}_.png")
    plot_pairs(points_xor,
               title=f'Puntos_XorShift_2D_{N}',
               save_dir=base_dir,
               filename=f"pairs_XorShift_{N}.png")
    plot_pairs(points_mt,
               title=f'Puntos_Mersenne_2D_{N}',
               save_dir=base_dir,
               filename=f"pairs_Mersenne_{N}.png")


    # Puntos 3D
    K = points * 3
    genCong = gen.genCongruencial(2605, 16807, 0, 2**31 - 1)
    genXor = gen.XorShift(2463534242)
    genMersenne = gen.Mersenne(5489)

    lcg_rng = [next(genCong) for _ in range(K)]
    xor_rng = [next(genXor) for _ in range(K)]
    mt_rng = [next(genMersenne) for _ in range(K)]

    points_lcg = generate_d_points(lcg_rng, 3)
    points_xor = generate_d_points(xor_rng, 3)
    points_mt  = generate_d_points(mt_rng, 3)

    plot_3d(points_lcg,
            title=f'Puntos_LCG_3D_{N}',
            save_dir=base_dir,
            filename=f"3d_LCG_{N}.png")
    plot_3d(points_xor,
            title=f'Puntos_XorShift_3D_{N}',
            save_dir=base_dir,
            filename=f"3d_XorShift_{N}.png")
    plot_3d(points_mt,
            title=f'Puntos_Mersenne_3D_{N}',
            save_dir=base_dir,
            filename=f"3d_Mersenne_{N}.png")

    